In [1]:
import pandas as pd

In [2]:
train_data=pd.read_csv('train classifier.csv')

In [84]:
train_data.head()

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe
0,explanationwhytheeditsmadeundermyusernamehardc...,explanationwhytheeditsmadeundermyusernamehardc...,0,0,0,0,0,0
1,dawwhematchesthisbackgroundcolouriamseeminglys...,dawwhematchesthisbackgroundcolouriamseeminglys...,0,0,0,0,0,0
2,heymaniamreallynottryingtoeditwaritsjustthatth...,heymaniamreallynottryingtoeditwaritsjustthatth...,0,0,0,0,0,0
3,moreicannotmakeanyrealsuggestionsonimprovement...,moreicannotmakeanyrealsuggestionsonimprovement...,0,0,0,0,0,0
4,yousiraremyheroanychanceyourememberwhatpagetha...,yousiraremyheroanychanceyourememberwhatpagetha...,0,0,0,0,0,0


In [4]:
train_data.shape

(159571, 8)

In [5]:
train_data['malignant'].value_counts()

0    144277
1     15294
Name: malignant, dtype: int64

# Data Cleaning
checking for duplicates and dropping them

In [6]:
train_data.duplicated().sum()

0

In [7]:
#dropping duplicate rows
train_data.drop_duplicates(inplace=True)

In [8]:
train_data.duplicated().sum()

0

# Checking for missing values

In [9]:
train_data.isnull().sum()

id                  0
comment_text        0
malignant           0
highly_malignant    0
rude                0
threat              0
abuse               0
loathe              0
dtype: int64

In [10]:
train_data.isnull().sum()*100/train_data.shape[0]

id                  0.0
comment_text        0.0
malignant           0.0
highly_malignant    0.0
rude                0.0
threat              0.0
abuse               0.0
loathe              0.0
dtype: float64

In [11]:
train_data['comment_text'].fillna('',inplace=True)
train_data['malignant'].fillna('',inplace=True)
train_data['highly_malignant'].fillna('missing',inplace=True)

In [12]:
train_data.isnull().sum()

id                  0
comment_text        0
malignant           0
highly_malignant    0
rude                0
threat              0
abuse               0
loathe              0
dtype: int64

# Data Preprocessing

In [13]:
import re
def decontracted(phrase):
    #specific
    phrase=re.sub(r" "," ",phrase)
    phrase=re.sub(r" "," ",phrase)
    phrase=re.sub(r" "," ",phrase)
    phrase=re.sub(r"won't","will not",phrase)
    phrase=re.sub(r"can\'t","can not",phrase)
    #general
    phrase=re.sub(r"n\'t","not",phrase)
    phrase=re.sub(r"\'re","are",phrase)
    phrase=re.sub(r"\'s","s",phrase)
    phrase=re.sub(r"\'d","would",phrase)
    phrase=re.sub(r"\'ll","will",phrase)
    phrase=re.sub(r"\'t","not",phrase)
    phrase=re.sub(r"\'ve","have",phrase)
    phrase=re.sub(r"\'m","am",phrase)
    return phrase
    

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
stop_words=stopwords.words('english')

In [18]:
lemmatizer=WordNetLemmatizer()

# comment_text

In [19]:
from tqdm import tqdm
preprocessed_comment_text=[]
#tqdm is for printing the status bar
for sentance in tqdm(train_data['comment_text'].values):
    sent=decontracted(sentance)
    sent=re.sub(r'https?:\/\/.*[\r\n]*','',sent)
    sent=re.sub('[^A-Za-z]+','',sent)
    sent=' '.join(e for e in sent.split() if e not in stop_words)
    sent=' '.join(lemmatizer.lemmatize(e) for e in sent.split())
    preprocessed_comment_text.append(sent.lower().strip())

100%|████████████████████████████████████████████████████████████████████████| 159571/159571 [00:24<00:00, 6458.49it/s]


In [20]:
train_data['comment_text']=preprocessed_comment_text

# id

In [21]:
from tqdm import tqdm
preprocessed_id=[]
#tqdm is for printing the status bar
for sentance in tqdm(train_data['comment_text'].values):
    sent=decontracted(sentance)
    sent=re.sub(r'https?:\/\/.*[\r\n]*','',sent)
    sent=re.sub('[^A-Za-z]+','',sent)
    sent=' '.join(e for e in sent.split() if e not in stop_words)
    sent=' '.join(lemmatizer.lemmatize(e) for e in sent.split())
    preprocessed_id.append(sent.lower().strip())

100%|███████████████████████████████████████████████████████████████████████| 159571/159571 [00:09<00:00, 17358.40it/s]


In [22]:
train_data['id']=preprocessed_id

# Splitting data into train,cv,test

In [23]:
from sklearn.model_selection import train_test_split

In [26]:
x==> indendent features
y==> target

SyntaxError: invalid syntax (<ipython-input-26-260e2c27d76d>, line 1)

In [27]:
output=train_data['malignant']

In [28]:
train,test,train_output,test_output=train_test_split(train_data.drop(columns={'malignant'}),output,test_size=0.3,stratify=output,random_state=0)
train,cv,train_output,cv_output=train_test_split(train,train_output,test_size=0.3,stratify=train_output,random_state=0)

In [29]:
train.shape,cv.shape,test.shape

((78189, 7), (33510, 7), (47872, 7))

In [30]:
train_output.shape,cv_output.shape,test_output.shape

((78189,), (33510,), (47872,))

# Data Encoding

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
comment_text_tfidf_vectorizer=TfidfVectorizer(min_df=5)
train_comment_text_tfidf=comment_text_tfidf_vectorizer.fit_transform(train['comment_text'].values)
cv_comment_text_tfidf=comment_text_tfidf_vectorizer.transform(cv['comment_text'].values)
test_comment_text_tfidf=comment_text_tfidf_vectorizer.transform(test['comment_text'].values)

In [33]:
#saving the tfidf vectorizer
import pickle
with open("comment_text_tfidf_vectorizer.pickle","wb") as fp:
    pickle.dump(comment_text_tfidf_vectorizer,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
comment_text_tfidf_vectorizer.get_feature_names()[:10]

C:\Users\HP\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['copyrightproblemremovedpriorcontentinthisarticleduplicatedoneormorepreviouslypublishedsourcesthematerialwascopiedfrom',
 'februaryutc',
 'julyutc',
 'junutc',
 'marutc',
 'novutc',
 'pleasedonotaddnonsensetowikipediaitisconsideredvandalismifyouwouldliketoexperimentusethesandboxthankyou',
 'pleasedonotaddnonsensetowikipediaitisconsideredvandalismifyouwouldliketoexperimentusethesandboxthankyoualinktotheeditihaverevertedcanbefoundherelinkifyoubelievethiseditshouldnothavebeenrevertedpleasecontactmetalk',
 'pleasedonotaddnonsensetowikipediaitisconsideredvandalismifyouwouldliketoexperimentusethesandboxthankyoutalk',
 'pleasestopifyoucontinuetovandalizepagesyouwillbeblockedfromeditingwikipedia']

# highly_malignant-Response coding

In [35]:
train['highly_malignant'].unique().shape

(2,)

In [36]:
prob_dict={}
train_highly_malignant=train.copy()
train_highly_malignant['malignant']=train_output
train_highly_malignant_1=train_highly_malignant.groupby('highly_malignant')
for i in (train_highly_malignant_1.groups):
    group=train_highly_malignant_1.get_group(i)
    tot=group.shape[0]
    fake=group[group['malignant']==1].shape[0]
    prob_fake=fake/tot
    prob_not_fake=1-prob_fake
    prob_dict.update({i:[prob_not_fake,prob_fake]})
keys=prob_dict.keys()
train_highly_malignant_response_code=[]
for highly_malignant in train['highly_malignant']:
    if highly_malignant not in keys:
        train_highly_malignant_response_code.append([0.5,0.5])
    else:
            train_highly_malignant_response_code.append(prob_dict.get(highly_malignant))
cv_highly_malignant_response_code=[]
for highly_malignant in cv['highly_malignant']:
    if highly_malignant not in keys:
        cv_highly_malignant_response_code.append([0.5,0.5])
    else:
            cv_highly_malignant_response_code.append(prob_dict.get(highly_malignant))

test_highly_malignant_response_code=[]
for author in test['highly_malignant']:
    if highly_malignant not in keys:
        test_highly_malignant_response_code.append([0.5,0.5])
    else:
        test_highly_malignant_response_code.append(prob_dict.get(highly_malignant))

In [37]:
#saving the probability dictionary
with open("prob_dict.pickle","wb") as fp:
    pickle.dump(prob_dict,fp,protocol=pickle.HIGHEST_PROTOCOL)

# id- TFIDF Vectorization

In [38]:
id_tfidf_vectorizer=TfidfVectorizer(min_df=5)
train_id_tfidf=id_tfidf_vectorizer.fit_transform(train['id'].values)
cv_id_tfidf=id_tfidf_vectorizer.transform(cv['id'].values)
test_id_tfidf=id_tfidf_vectorizer.transform(test['id'].values)

In [39]:
with open("id_tfidf_vectorizer.pickle","wb") as fp:
    pickle.dump(id_tfidf_vectorizer,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [40]:
id_tfidf_vectorizer.get_feature_names()[:10]

C:\Users\HP\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['copyrightproblemremovedpriorcontentinthisarticleduplicatedoneormorepreviouslypublishedsourcesthematerialwascopiedfrom',
 'februaryutc',
 'julyutc',
 'junutc',
 'marutc',
 'novutc',
 'pleasedonotaddnonsensetowikipediaitisconsideredvandalismifyouwouldliketoexperimentusethesandboxthankyou',
 'pleasedonotaddnonsensetowikipediaitisconsideredvandalismifyouwouldliketoexperimentusethesandboxthankyoualinktotheeditihaverevertedcanbefoundherelinkifyoubelievethiseditshouldnothavebeenrevertedpleasecontactmetalk',
 'pleasedonotaddnonsensetowikipediaitisconsideredvandalismifyouwouldliketoexperimentusethesandboxthankyoutalk',
 'pleasestopifyoucontinuetovandalizepagesyouwillbeblockedfromeditingwikipedia']

# Word2Vec

In [41]:
from gensim.models import Word2Vec

In [42]:
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f=open(gloveFile,'r',errors='ignore',encoding='utf8')
    model={}
    for line in tqdm(f):
        splitLine=line.split()
        word=splitLine[0]
        embedding=np.asarray([float(val) for val in splitLine[1:]])
        model[word]=embedding
    print ("Done.",len(model),"words loades!")
    return model


In [43]:
with open("prob_dict.pickle","wb") as fp:
    pickle.dump(prob_dict,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
glove_words=set()

In [73]:
avg_w2v_vectors_comment_text_test=train

# comment_text-Word2Vec

In [74]:
avg_w2v_vectors_comment_text_train=[];
avg_w2v_vectors_comment_text_cv=[];
avg_w2c_vectors_comment_text_test=[];

for sentance in tqdm(train['comment_text'].values):
    vector=np.zeros(300)
    cnt_words=0;
    for word in sentance:
        if word in glove_words:
            vector += model[word]
            cnt_words +=1
    if cnt_words !=0:
        vector /=cnt_words
    avg_w2v_vectors_comment_text_train.append(vector)
avg_w2v_vectors_comment_text_train=np.array(avg_w2v_vectors_comment_text_train)

for sentance in tqdm(cv['comment_text'].values):
    vector=np.zeros(300)
    cnt_words=0;
    for word in sentance:
        if word in glove_words:
            vector +=model[word]
            cnt_words +=1
        if cnt_words !=0:
            vector /=cnt_words
        avg_w2v_vectors_comment_text_cv.append(vector)
avg_w2v_vectors_comment_text_cv=np.array(avg_w2v_vectors_comment_text_cv)

for sentance in tqdm(test['comment_text'].values):
    vectors=np.zeros(300)
    cnt_words=0;
    for word in sentance:
        if word in glove_words:
            vector += model[word]
            cnt_words +=1
    if cnt_words !=0:
        vector /=cnt_words
    avg_w2v_vectors_comment_text_test.append(vector)
avg_w2v_vectors_comment_text_test=np.array(avg_w2v_vectors_comment_text_test)
            



  0%|                                                                                        | 0/47872 [00:00<?, ?it/s]


TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

# combining all encoded features

In [46]:
from scipy.sparse import hstack

# combining TFIDF encoded features

In [87]:
train_id_response_code=train
train_text_tfidf=train
train_id_tfidf=train

In [88]:
train_data_final_tfidf=hstack((train_id_tfidf,train_id_response_code,train_text_tfidf))
cv_data_final_tfidf=hstack((cv_id_tfidf,cv_id_response_code,cv_text_tfidf))
test_data_final_tfidf=hstack((test_id_tfidf,test_id_response_code,test_text_tfidf))

ValueError: blocks must be 2-D

In [ ]:
train_data_final_tfidf.shape,cv_data_final_tfidf.shape,test_data_final_tfidf.shape

# combining word2ves encoded features

In [65]:
avg_w2v_vectors_id_train=train
cv_id_response_code=train
avg_w2v_vectors_text_cv=train

In [66]:
train_data_final_w2v=np.concatenate((avg_w2v_vectors_comment_text_train,np.array(train_id_response_code),avg_w2v_vectors_id_train),axis=1)
cv_data_final_w2v=np.concatenate((avg_w2v_vectors_comment_text_cv,np.array(cv_id_response_code),avg_w2v_vectors_text_cv),axis=1)
test_data_final_w2v=np.concatenate((avg_w2v_vectors_comment_text_test,np.array(test_id_response_code),avg_w2v_vectors_id_test),axis=1)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 10186666 and the array at index 1 has size 78189

In [67]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
train_data_final_w2v=scaler.fit_transform(train_data_final_w2v)
cv_data_final_w2v=scaler.transform(cv_data_final_w2v)
test_data_final_w2v=scaler.transform(test_data_final_w2v)

ValueError: could not convert string to float: 'mayutcgiventhenatureofdreamhostitcouldbeverydamagingtobeassociatedwiththatcompanyinanywayiaskthatyouceasemessingwiththisarticleyourpersonalattacksandbsaresimplytooextremewhatsortofcowardareyouanywayattackingpeopleontheweb'

In [68]:
with open("prob_dict.pickle","wb") as fp:
    pickle.dump(prob_dict,fp,protocol=pickle.HIGHEST_PROTOCOL)

# Modelling

In [69]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,f1_score,classification_report

# Multinomail Naive Bayes with TFIDF encoded features

In [71]:
train_data_final_tfidf=train

In [72]:
alpha_range=[0.00001,0.0001,0.01,0.1,1,10,100]
for i in alpha_range:
    nb_clf=MultinomialNB(alpha=i,fit_prior=True)
    nb_clf.fit(train_data_final_tfidf,train_output)
    
    train_prob=nb_clf.predict_proba(train_data_final_tfidf)[:,1]
    train_AUC=roc_auc_score(train_output,train_prob)
    print("For alpha=%f,train AUC=%f" % (i,train_AUC))
    
    cv_prob=nb_clf.predict_proba(cv_data_final_tfidf)[:,1]
    cv_AUC=roc_auc_score(cv_output,train_prob)
    print("For alpha=%f,cv AUC=%f" % (i,cv_AUC))
    
    train_score=nb_clf.predict(train_data_final_tfidf)
    train_f1=f1_score(train_output,train_scores)
    print("For alpha=%f,train f1 score=%f" % (i,train_f1))
    
    cv_score=nb_clf.predict(cv_data_final_tfidf)
    cv_f1=f1_score(cv_output,cv_scores)
    print("For alpha=%f,cv f1 score=%f" % (i,cv_f1))
    
    print("-"*50)

ValueError: could not convert string to float: 'mayutcgiventhenatureofdreamhostitcouldbeverydamagingtobeassociatedwiththatcompanyinanywayiaskthatyouceasemessingwiththisarticleyourpersonalattacksandbsaresimplytooextremewhatsortofcowardareyouanywayattackingpeopleontheweb'